In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

import os

# Specify the folder path
model_folder_path = '/content/gdrive/MyDrive/Colab Notebooks/MOR - Safety/Version2/faulty_wire_model_V2'
img_folder_path = '/content/gdrive/MyDrive/Colab Notebooks/MOR - Safety/Version2/all'

# List all files in the folder
os.listdir(model_folder_path)
os.listdir(img_folder_path)


Mounted at /content/gdrive


['Copy of IS_67.png',
 'Copy of IS_71.png',
 'Copy of IS_70.png',
 'Copy of IS_65.png',
 'Copy of IS_66.png',
 'Copy of IS_68.png',
 'Copy of IS_82.png',
 'Copy of LW_22.png',
 'Copy of LW_28.png',
 'Copy of LW_21.png',
 'Copy of LW_27.png',
 'Copy of LW_29.png',
 'Copy of LW_34.png',
 'Copy of LW_35.png',
 'Copy of LW_39.png',
 'Copy of LW_32.png',
 'Copy of LW_40.png',
 'Copy of LW_42.png',
 'Copy of LW_48.png',
 'Copy of LW_41.png',
 'Copy of LW_53.png',
 'Copy of LW_51.png',
 'Copy of LW_74.png',
 'Copy of LW_76.png',
 'Copy of LW_73.png',
 'Copy of LW_85.png',
 'Copy of LW_80.png',
 'Copy of LW_82.png',
 'Copy of LW_89.png',
 'Copy of LW_86.png',
 'Copy of LW_88.png',
 'Copy of LW_84.png',
 'Copy of LW_81.png',
 'Copy of LW_91.png',
 'Copy of LW_93.png',
 'Copy of LW_100.png',
 'Copy of LW_92.png',
 'Copy of LW_101.png',
 'Copy of LW_109.png',
 'Copy of LW_114.png',
 'Copy of LW_111.png',
 'Copy of LW_106.png',
 'Copy of LW_108.png',
 'Copy of LW_104.png',
 'Copy of LW_126.png',
 

In [4]:
# Import packages
import os
import cv2
import numpy as np
import sys
import glob
import importlib.util
from threading import Thread
from google.colab.patches import cv2_imshow

# Configure Variables
MODEL_NAME = '/content/gdrive/MyDrive/Colab Notebooks/MOR - Safety/Version2/faulty_wire_model_V2'
GRAPH_NAME = 'detect.tflite'
LABELMAP_NAME = 'labelmap.txt'

min_conf_threshold = 0.5
use_TPU = False

save_results = False  # Defaults to False
show_results = True  # Defaults to True

# Import TensorFlow libraries
import importlib.util

use_TPU = False  # Set this to True if you want to use the TPU delegate
try:
    pkg = importlib.util.find_spec('tflite_runtime')
    if pkg:
        from tflite_runtime.interpreter import Interpreter
        if use_TPU:
            from tflite_runtime.interpreter import load_delegate
    else:
        raise ImportError  # Fall back to TensorFlow
except ImportError:
    try:
        from tensorflow.lite.python.interpreter import Interpreter
        if use_TPU:
            from tensorflow.lite.python.interpreter import load_delegate
    except ImportError as e:
        raise ImportError("Neither tflite_runtime nor TensorFlow Lite is available. Please install one of them.") from e

# If using Edge TPU, adjust the graph name
if use_TPU and (GRAPH_NAME == 'detect.tflite'):
    GRAPH_NAME = 'edgetpu.tflite'

# Get path to current working directory
CWD_PATH = os.getcwd()

# Define path to images and grab all image filenames
PATH_TO_IMAGES = os.path.join(CWD_PATH, IM_NAME) if IM_NAME else CWD_PATH
images = glob.glob(PATH_TO_IMAGES)

# Create results directory if user wants to save results
if save_results:
    RESULTS_DIR = 'results'
    RESULTS_PATH = os.path.join(CWD_PATH, RESULTS_DIR)
    os.makedirs(RESULTS_PATH, exist_ok=True)

# Path to .tflite file, which contains the model that is used for object detection
PATH_TO_CKPT = os.path.join(CWD_PATH, MODEL_NAME, GRAPH_NAME)

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH, MODEL_NAME, LABELMAP_NAME)

# Load the label map
with open(PATH_TO_LABELS, 'r') as f:
    labels = [line.strip() for line in f.readlines()]

# Remove first label if using the COCO starter model
if labels[0] == '???':
    del (labels[0])

# Load the TensorFlow Lite model
if use_TPU:
    interpreter = Interpreter(model_path=PATH_TO_CKPT, experimental_delegates=[load_delegate('libedgetpu.so.1.0')])
else:
    interpreter = Interpreter(model_path=PATH_TO_CKPT)

interpreter.allocate_tensors()

# Get model details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

floating_model = (input_details[0]['dtype'] == np.float32)

input_mean = 127.5
input_std = 127.5

# Determine output indices based on TensorFlow version
outname = output_details[0]['name']
if 'StatefulPartitionedCall' in outname:
    boxes_idx, classes_idx, scores_idx = 1, 3, 0
else:
    boxes_idx, classes_idx, scores_idx = 0, 1, 2

# Loop over every image and perform detection
for image_path in images:

    # Load image and resize to expected shape [1xHxWx3]
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    imH, imW, _ = image.shape
    image_resized = cv2.resize(image_rgb, (width, height))
    input_data = np.expand_dims(image_resized, axis=0)

    # Normalize pixel values if using a floating model
    if floating_model:
        input_data = (np.float32(input_data) - input_mean) / input_std

    # Perform the detection
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    # Retrieve detection results
    boxes = interpreter.get_tensor(output_details[boxes_idx]['index'])[0]
    classes = interpreter.get_tensor(output_details[classes_idx]['index'])[0]
    scores = interpreter.get_tensor(output_details[scores_idx]['index'])[0]

    detections = []

    # Process each detection
    for i in range(len(scores)):
        if ((scores[i] > min_conf_threshold) and (scores[i] <= 1.0)):
            # Get bounding box coordinates
            ymin = int(max(1, (boxes[i][0] * imH)))
            xmin = int(max(1, (boxes[i][1] * imW)))
            ymax = int(min(imH, (boxes[i][2] * imH)))
            xmax = int(min(imW, (boxes[i][3] * imW)))

            cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (10, 255, 0), 2)

            # Draw label
            object_name = labels[int(classes[i])]
            label = '%s: %d%%' % (object_name, int(scores[i] * 100))
            labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
            label_ymin = max(ymin, labelSize[1] + 10)
            cv2.rectangle(image, (xmin, label_ymin - labelSize[1] - 10),
                          (xmin + labelSize[0], label_ymin + baseLine - 10),
                          (255, 255, 255), cv2.FILLED)
            cv2.putText(image, label, (xmin, label_ymin - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

            detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])

   # Display the image
if show_results:
    cv2_imshow(image)  # Use cv2_imshow instead of cv2.imshow

# Save the labeled image if desired
if save_results:
    image_fn = os.path.basename(image_path)
    image_savepath = os.path.join(RESULTS_PATH, image_fn)

    base_fn, ext = os.path.splitext(image_fn)
    txt_result_fn = base_fn + '.txt'
    txt_savepath = os.path.join(RESULTS_PATH, txt_result_fn)

    # Save image
    cv2.imwrite(image_savepath, image)

    # Write results to text file
    with open(txt_savepath, 'w') as f:
        for detection in detections:
            f.write('%s %.4f %d %d %d %d' % (
            detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))

# Clean up
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.

In [1]:
IM_NAME = '/content/img1.png'

In [ ]:
x_train = [38, 171]
y_train = [100, 210]

x_pred = [34, 171]
y_pred = [98, 212]

# Calculate area of ground truth and prediction
area_train = (x_train[1] - x_train[0]) * (y_train[1] - y_train[0])
area_pred = (x_pred[1] - x_pred[0]) * (y_pred[1] - y_pred[0])

# Calculate the intersection (overlapping area)
overlap_width = max(0, min(x_train[1], x_pred[1]) - max(x_train[0], x_pred[0]))
overlap_height = max(0, min(y_train[1], y_pred[1]) - max(y_train[0], y_pred[0]))
area_overlap = overlap_width * overlap_height

# Calculate the union area
area_union = area_train + area_pred - area_overlap

# Compute mAP (IoU)
mAP = area_overlap / area_union if area_union > 0 else 0  # Avoid division by zero
mAP = round(mAP * 100, 3)

print(mAP)

93.674
